In [ ]:
from petastorm import make_reader
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import os
sys.path.append(os.path.join(os.getcwd(), ".."))
from Metrics import Wrapper_main_function
path_formatted_glasgow = "/workspaces/maitrise/data/20221006_physio_quality/set-a/dataParquet"
path_dataset= f"file:///{path_formatted_glasgow}"
path_csv_ref_label = "/workspaces/maitrise/data/20221006_physio_quality/set-a/REFERENCE.csv"

In [ ]:
label_ref = pd.read_csv(path_csv_ref_label)
label_ref = label_ref.to_numpy()
Y = label_ref[:,1].copy()
Y_true = Y[Y.copy()!="unlabeled"]
X_true = label_ref[:,0].copy()
X_true = X_true[Y!="unlabeled"].astype(int)

### First load your dataset (output variables : numpy array [n_patients,n_leads,signal_length])


In [ ]:

len_patients = 998 ## patients have undetermined label for the 2011 physionet dataset
len_lead = 12
Data = np.empty([len_patients,len_lead,5000])
index_patient  = np.array([])
ind = 0

with make_reader(path_dataset) as reader:
    for sample in reader:
        data = sample
        if data.signal_quality == "unlabeled".encode():
            continue
        else :
            ECG_lead = sample.signal_names
            fs = sample.sampling_frequency
            status = int(sample.noun_id)
            index_patient = np.append(index_patient,status)
            dico_ECG = np.zeros([len(ECG_lead),sample.signal.shape[0]])
            for i in range(len(sample.signal_names)):
                dico_ECG[i,:] = sample.signal[:,i]
            Data[ind,:,:] = dico_ECG
            ind += 1

Data = Data[np.argsort(index_patient),:,:]##ordered your data with your label

### Second : Use the main function to get the feautres for each of your patient)
### Output : numpy array [n_patients,n_lead,n_features]

In [ ]:
name_method = ["Corr_interlead","Corr_intralead"]
Matrix_features = np.empty([len(index_patient),len_lead,len(name_method)])

for j in range(Matrix_features.shape[0]):
    Patient = Data[j,:,:]
    Matrix_features[j,:,:] = Wrapper_main_function.main(Patient,fs,name_method)
